In [1]:
import json
import os
import os.path as osp
import sys
from PIL import Image
import pickle as pkl
import cv2

from detectron.datasets.json_dataset import JsonDataset
from detectron.utils.io import load_object, save_object
from detectron.datasets.roidb import combined_roidb_for_training
import detectron.utils.vis as vis_utils
from detectron.core.config import cfg


def checkMkdir(dirname):
    if not osp.isdir(dirname):
        os.makedirs(dirname)
        
dataset = 'furniture_train'
boxFilePath = '/media/data/chnxi/GOD/threshold_bboxes/'
mapFilePath = '/media/data/chnxi/GOD/mappings/src_nameid_to_GOD_id_mappings/'
outAnnoFolder = '/media/data/chnxi/GOD/json_annotations/'
outAnnoFile = osp.join(outAnnoFolder, 'tmp_{}_annotations.json'.format(dataset))

visPath = '/media/data/chnxi/GOD/visualizations/'
visFolder = osp.join(visPath, dataset)
checkMkdir(visFolder)

modelSetMap = {'Fashion_Detector'  : 'fashion',
               'HF_Detector'       : 'furniture',
               'COCO_Mask_Detector': 'COCO',
               'OpenImage_Detector': 'Open Image'}

if dataset == 'furniture_train':
    gtSetName = 'furniture'
    mapFile = mapFilePath + 'HF_dataset_sources_ids.pkl'
    srcFiles = {'Fashion_Detector'  : 'furniture_train.json',
                'COCO_Mask_Detector': 'furniture_train.pkl',
                'OpenImage_Detector': 'furniture_train.pkl'}
elif dataset == 'FashionV2_train':
    gtSetName = 'fashion'
    mapFile = mapFilePath + 'Fashion_dataset_sources_ids.pkl'
    srcFiles = {'HF_Detector'       : 'FashionV2_train.pkl',
                'COCO_Mask_Detector': 'FashionV2_train.pkl',
                'OpenImage_Detector': 'FashionV2_train.pkl'}
elif dataset == 'OpenImage_train':
    gtSetName = 'Open Image'
    mapFile = mapFilePath + 'OpenImage_dataset_sources_ids.pkl'
    srcFiles = {'Fashion_Detector'  : 'OpenImage_train.json',
                'HF_Detector'       : 'OpenImage_train.pkl',
                'COCO_Mask_Detector': 'OpenImage_train.pkl'}
elif dataset == 'COCO_trainval':
    gtSetName = 'COCO'
    mapFile = mapFilePath + 'COCO_dataset_sources_ids.pkl'
    srcFiles = {'Fashion_Detector'  : 'COCO_trainval.json',
                'HF_Detector'       : 'coco_2014_trainval.pkl',
                'OpenImage_Detector': 'COCO_trainval.pkl'}

# for modelName in srcFiles:
#     box_file_name = srcFiles[modelName]
#     boxFileName = osp.join(boxFilePath, modelName, box_file_name)
#     print ("boxFileName = {}".format(boxFileName))
    
    
    

In [2]:
if dataset in ['furniture_train', 'FashionV2_train', 'OpenImage_train']:
    print ("creating json dataset {}".format(dataset))
    ds = JsonDataset(dataset)
    roidb = ds.get_roidb(gt=True)
elif dataset == 'COCO_trainval':
    datasetNames = ('coco_2014_train','coco_2014_valminusminival')
    roidb = []
    for dsName in datasetNames:
        ds = JsonDataset(dsName)
        rdb = ds.get_roidb(gt=True)
        roidb = roidb + rdb
imgNum = len(roidb)
print ('imgNum = {}'.format(imgNum))

creating json dataset furniture_train
================= loading annotation furniture_train
loading annotations into memory...
Done (t=7.20s)
creating index...
index created!
getting roidb
Done (t=95.80s)
imgNum = 127068


In [3]:
print (roidb[0]['image'])
print (roidb[-1]['image'])

/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/HomeFurniture/Images/17B0E85E02491E313EF2D908471730BD98FC5D553368E3F7A9F2AB3C25FF05FDB6C24FB7B7A1A7066AE04A2909ADB593E1925D7A6079E81444A3B3E9F1E4A8E3.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/HomeFurniture/Images/CE29382FB5DA946DC2BB68A1AF5A01A1FF28D472D3432A0F6FF0807DAD887B38BEC061FED7A3C4892FDB514AE66C0638E50C3284DCE37CB6FC788CC906A4CD8F.jpg


In [4]:
print ("loading {}".format(mapFile))
src_god_map = pkl.load(open(mapFile,'rb'))
print (src_god_map.keys())
cocoMap = src_god_map['name_to_god_id']['COCO']
print (cocoMap)
cocoIdMap = src_god_map['id_map']['COCO']
print(cocoIdMap)
print (len(cocoMap))

loading /media/data/chnxi/GOD/mappings/src_nameid_to_GOD_id_mappings/HF_dataset_sources_ids.pkl
dict_keys(['id_map', 'name_to_god_id'])
{'teddy bear': 541, 'bicycle': 546, 'apple': 281, 'laptop': 112, 'tennis racket': 514, 'donut': 263, 'snowboard': 509, 'carrot': 287, 'zebra': 82, 'keyboard': 109, 'scissors': 478, 'fire hydrant': 245, 'mouse': 111, 'airplane': 542, 'boat': 564, 'kite': 499, 'sheep': 77, 'horse': 58, 'cup': 338, 'traffic light': 570, 'toaster': 140, 'bowl': 334, 'book': 464, 'elephant': 51, 'orange': 296, 'stop sign': 568, 'knife': 342, 'pizza': 275, 'fork': 340, 'hair drier': 129, 'sandwich': 278, 'umbrella': 237, 'banana': 284, 'bus': 547, 'toothbrush': 407, 'spoon': 352, 'giraffe': 53, 'cell phone': 148, 'broccoli': 285, 'frisbee': 492, 'remote': 153, 'surfboard': 512, 'cow': 47, 'hot dog': 268, 'car': 548, 'skateboard': 507, 'dog': 49, 'cat': 45, 'baseball glove': 488, 'skis': 508, 'train': 559, 'truck': 553, 'parking meter': 453, 'suitcase': 161, 'cake': 251, 'win

In [5]:
print ("Loading model bboxes!")
srcBoxes = {}
for modelName in srcFiles:
    box_file_name = srcFiles[modelName]
    boxFileName = osp.join(boxFilePath, modelName, box_file_name)
    print ("Loading boxFileName = {}".format(boxFileName))
    if modelName in ['COCO_Mask_Detector', 'HF_Detector']:
        # format: all_boxes[imgid][class_id]
        all_boxes = load_object(boxFileName)
        print ("loaded")
        print ("all_boxes.size = ({}, {})".format(len(all_boxes), len(all_boxes[0])))
        srcBoxes[modelName] = all_boxes
    elif modelName == 'OpenImage_Detector':
        # all_boxes[file_name] = {'file_name': file_name,
        #        'boxes': boxes,
        #        'label_names': label_names,
        #        'labels': labels}
        all_boxes = load_object(boxFileName)
        print ("len(all_boxes) = {}".format(len(all_boxes)))
        srcBoxes[modelName] = all_boxes
    elif '.json' in box_file_name:
        # {'image_id': 'd7a0b91d2d83d457', 
        #'boxes': [{'score': 0.228, 'category_id': 2, 
        #'bbox': [506.49920000000003, 414.4128, 657.8248, 544.1536], 
        #'category_name': 'accessories-glasses'}], 'image_file_name': 'd7a0b91d2d83d457.jpg'}
        print ("loading {}".format(boxFileName))
        dets = json.load(open(boxFileName,'r'))
        all_boxes = {}
        for ix, det in enumerate(dets):
            # make it a dictionary with image file name as key
            img_file_name = det['image_file_name']
            all_boxes[img_file_name] = det['boxes']
        print ("len(all_boxes) = {}".format(len(all_boxes)))
        print ("loaded")
        srcBoxes[modelName] = all_boxes #dets
    print ("srcBoxes[{}] loaded".format(modelName))

Loading model bboxes!
Loading boxFileName = /media/data/chnxi/GOD/threshold_bboxes/Fashion_Detector/furniture_train.json
loading /media/data/chnxi/GOD/threshold_bboxes/Fashion_Detector/furniture_train.json
len(all_boxes) = 79277
loaded
srcBoxes[Fashion_Detector] loaded
Loading boxFileName = /media/data/chnxi/GOD/threshold_bboxes/COCO_Mask_Detector/furniture_train.pkl
loaded
all_boxes.size = (127068, 80)
srcBoxes[COCO_Mask_Detector] loaded
Loading boxFileName = /media/data/chnxi/GOD/threshold_bboxes/OpenImage_Detector/furniture_train.pkl
len(all_boxes) = 127068
srcBoxes[OpenImage_Detector] loaded


In [6]:
imgName = '2988531ACD30C78DC4C1F1EA859ED1C02DDF7773F9CBC67FFAD71F25A2CAD0D688A35E26D72C7525A0C4100ED0C9C3CA008942A59F6B9173D1E3552387B19AF3.jpg'
print ("srcBoxes['COCO_Mask_Detector'][10][14][1] = {}\n".format(srcBoxes['COCO_Mask_Detector'][10][14][1]))
print ("srcBoxes['Fashion_Detector'][imgName] = {}\n".format(srcBoxes['Fashion_Detector'][imgName]))
print ("srcBoxes['OpenImage_Detector'][imgName] = {}\n".format(srcBoxes['OpenImage_Detector'][imgName]))

srcBoxes['COCO_Mask_Detector'][10][14][1] = [279.27884   59.576763 296.89883   74.03485    0.592858]

srcBoxes['Fashion_Detector'][imgName] = [{'score': 0.5268, 'category_id': 48, 'bbox': [423.63, 761.9339, 475.65, 829.0], 'category_name': 'shoes-boots'}, {'score': 0.5247, 'category_id': 48, 'bbox': [593.01, 177.9034, 633.87, 235.7676], 'category_name': 'shoes-boots'}]

srcBoxes['OpenImage_Detector'][imgName] = {'file_name': '2988531ACD30C78DC4C1F1EA859ED1C02DDF7773F9CBC67FFAD71F25A2CAD0D688A35E26D72C7525A0C4100ED0C9C3CA008942A59F6B9173D1E3552387B19AF3.jpg', 'boxes': array([], dtype=float64), 'label_names': [], 'labels': array([], dtype=float64)}



In [7]:
#############   Creating taxonomy   #############################################
import pandas as pd
labelFile = '/media/data/chnxi/GOD/taxonomy/GOD_taxonomy_V1.tsv'
god_data = pd.read_csv(labelFile, delimiter='\t')
god_name_id_map = dict(zip(god_data['GOD_v1_name'], god_data['GOD_v1_id']))
god_classes = list(god_data['GOD_v1_name'])
print ("len(god_classes) original = {}".format(len(god_classes)))
god_classes = ['__background__'] + god_classes

len(god_classes) original = 579


In [8]:
data = {}
supercat = 'Generic'
data['info'] = {'description':'Bing Generic Object Detection dataset on {}'.format(dataset)}
data['type'] = 'instances'

data['categories'] = []

for cls, id in god_name_id_map.items():
    cats = {'supercategory':supercat,
            'id':id,
            'name':cls}
    data['categories'].append(cats)

print (data['categories'][:10])

[{'supercategory': 'Generic', 'id': 1, 'name': 'animal\\amphibian\\frog'}, {'supercategory': 'Generic', 'id': 2, 'name': 'animal\\arthropod\\arachnid\\scorpion'}, {'supercategory': 'Generic', 'id': 3, 'name': 'animal\\arthropod\\arachnid\\spider'}, {'supercategory': 'Generic', 'id': 4, 'name': 'animal\\arthropod\\crustacean\\crab'}, {'supercategory': 'Generic', 'id': 5, 'name': 'animal\\arthropod\\crustacean\\lobster'}, {'supercategory': 'Generic', 'id': 6, 'name': 'animal\\arthropod\\crustacean\\shrimp'}, {'supercategory': 'Generic', 'id': 7, 'name': 'animal\\arthropod\\insect\\ant'}, {'supercategory': 'Generic', 'id': 8, 'name': 'animal\\arthropod\\insect\\bee'}, {'supercategory': 'Generic', 'id': 9, 'name': 'animal\\arthropod\\insect\\butterfly'}, {'supercategory': 'Generic', 'id': 10, 'name': 'animal\\arthropod\\insect\\caterpillar'}]


In [9]:
imgNum = len(roidb)
# imgIds = range(imgNum)

In [10]:
import matplotlib.pyplot as plt
color_list = ['g', 'r', 'b', 'm']
def visImageClsThrshs(plt, splot, im, imgBoxes, ecolor='m', title='', showCateg=False):
    #plt.cla()
    fx, fy, fi = splot
    plt.subplot(fx, fy, fi)
    plt.imshow(im)
    plt.axis('off')
    plt.title(title)
    ax = plt.gca()
    for obj in imgBoxes:
        #print obj
        bbox = obj['bbox']
        categ_id = obj['category_id']
        categ = god_classes[categ_id].split('\\')[-1]
        #ecolor = 'm' #if gid == jmax else gtcolor
        #print ("{}: bbox = {}".format(categ, bbox))
        ax.add_patch(
            plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2],
                          bbox[3], fill=False,
                          edgecolor=ecolor, linestyle='-', linewidth=1))
        if showCateg:
            ax.text(bbox[0], bbox[1]-2, '{}'.format(categ), bbox={'facecolor': ecolor, 'alpha': 0.5})
    plt.draw()


def getAnno(image_id, bbox, categ_id, box_id):
    boxw = float(bbox[2]-bbox[0]) # convert np.ndarray.float32 to float in case json.dump not serializable
    boxh = float(bbox[3]-bbox[1])
    nbbox = [float(bbox[0]), float(bbox[1]), boxw, boxh]
    anno = {'segmentation':[],
            'area': boxw*boxh,
            'iscrowd':0,
            'image_id':image_id,
            'bbox':nbbox, # because COCO format [l, t, w,h] #float(r),float(b)],
            'category_id':categ_id,
            'id':box_id
            }
#     print ("anno.segmentation.type = {}".format(type(anno['segmentation'])))
#     print ("anno.bbox.type = {}".format(type(anno['bbox'])))
#     print ("anno.area.type = {}".format(type(anno['area'])))
#     print ("anno.iscrowd.type = {}".format(type(anno['iscrowd'])))
#     print ("anno.image_id.type = {}".format(type(anno['image_id'])))
#     print ("anno.category_id.type = {}".format(type(anno['category_id'])))
#     print ("anno.id.type = {}".format(type(anno['id'])))
    return anno

# from random import randint
# dsIdx = randint(0,imgNum-1)
# print(dsIdx)
# imgIds = [dsIdx]
# print (imgIds)

import random
visNum = 200
imgIds = random.sample(range(len(roidb)), visNum)
print(imgIds)

vis = True
saveVis = True
totalBoxCnt = 0
data["images"] = []
data["annotations"] = []

for imgid in imgIds: #imgid, entry in enumerate(roidb):
    visAnno = {} 
    entry = roidb[imgid]
    ###############################################
    ## data['images']
    ###############################################
    file_name = entry['image']
    imgName = osp.basename(file_name)
    imgH = entry['height']
    imgW = entry['width']
    img = {'file_name': imgName,
           'height': float(entry['height']),
           'width': float(entry['width']),
           'id':imgid
           }
    data["images"].append(img)
    print (img)
    print ("img.file_name.type = {}".format(type(img['file_name'])))
    print ("img.height.type = {}".format(type(img['height'])))
    print ("img.width.type = {}".format(type(img['width'])))
    print ("img.id.type = {}".format(type(img['id'])))
    ###############################################
    ## Add GT Boxes
    ###############################################
    src_god_id_map = src_god_map['id_map'][gtSetName]
    src_name_godid_map = src_god_map['name_to_god_id'][gtSetName]
    visName = gtSetName + '_GT'
    visAnno[visName] = []
    gt_boxes = entry['boxes']
    gt_src_cls_ids = entry['gt_classes']
    for bid, box in enumerate(gt_boxes):
        src_class_id = gt_src_cls_ids[bid]
        if src_class_id in src_god_id_map:
            god_class_id = src_god_id_map[src_class_id]
            print ("======{}:{} ==> GOD:{} {}".format(gtSetName, src_class_id, god_class_id, god_classes[god_class_id]))
            anno = getAnno(imgid, box[:4], god_class_id, totalBoxCnt)
            #print (anno)
            data["annotations"].append(anno)
            if vis:
                visAnno[visName].append(anno)
            totalBoxCnt += 1
    ###############################################
    ## Add boxes from other models
    ###############################################
    for modelName in srcFiles:
        modelSetName = modelSetMap[modelName]
        src_god_id_map = src_god_map['id_map'][modelSetName]
        src_name_godid_map = src_god_map['name_to_god_id'][modelSetName]
        visAnno[modelName] = []
#         print ("{} ==> {}".format(modelName, modelSetName))
        if modelName in ['COCO_Mask_Detector', 'HF_Detector']:
            # format: all_boxes[imgid][class_id]
            for cid, cls_boxes in enumerate(srcBoxes[modelName][imgid]):
                src_class_id = cid + 1 # all_boxes starts from 0
                if src_class_id in src_god_id_map and len(cls_boxes) > 0:
                    god_class_id = src_god_id_map[src_class_id]
                    print ("========{}:{} ==> GOD:{}:{}".format(modelSetName, src_class_id, god_class_id, god_classes[god_class_id]))
                    for bbox in cls_boxes:
                        anno = getAnno(imgid, bbox, god_class_id, totalBoxCnt)
                        #if totalBoxCnt % 10 == 0:
                        #print (anno)
                        data["annotations"].append(anno)
                        if vis:
                            visAnno[modelName].append(anno)
                        totalBoxCnt += 1
        if modelName == 'OpenImage_Detector':
            # all_boxes[file_name] = {'file_name': file_name,
            #        'boxes': boxes,
            #        'label_names': label_names,
            #        'labels': labels}
            boxes = srcBoxes[modelName][imgName]['boxes']
            label_names = srcBoxes[modelName][imgName]['label_names']
            for bid, box in enumerate(boxes):
                src_class_name = label_names[bid] # do not use labels: it's not correct for OpenImage Id
                if src_class_name in src_name_godid_map:
                    god_class_id = src_name_godid_map[src_class_name]
                    print ("========= {}:{} ==> GOD:{}:{}".format(modelSetName, src_class_name, god_class_id, god_classes[god_class_id]))
                    bbox = [box[0] * imgW, box[1] * imgH, box[2] * imgW, box[3] * imgH]
                    anno = getAnno(imgid, bbox, god_class_id, totalBoxCnt)
                    #print (anno)
                    data["annotations"].append(anno)
                    if vis:
                        visAnno[modelName].append(anno) 
                    totalBoxCnt += 1                 
        if modelName == 'Fashion_Detector': #'.json' in box_file_name:
            # all_boxes[imgName] = [ {[506.49920000000003, 414.4128, 657.8248, 544.1536], 
            #'category_name': 'accessories-glasses'} ]
            if imgName in srcBoxes[modelName]:
                boxes = srcBoxes[modelName][imgName]
                for bid, obj in enumerate(boxes):
                    src_class_name = obj['category_name']
                    if src_class_name in src_name_godid_map:
                        print (obj)
                        god_class_id = src_name_godid_map[src_class_name]
                        print ("========{}:{} ==> GOD:{}{}".format(modelSetName, src_class_name, god_class_id, god_classes[god_class_id]))
                        bbox = obj['bbox']
                        anno = getAnno(imgid, bbox, god_class_id, totalBoxCnt)
                        #print (anno)
                        data["annotations"].append(anno)
                        if vis:
                            visAnno[modelName].append(anno) 
                        totalBoxCnt += 1    

    #################################
    # visualization of merging boxes
    #################################
    if vis:
        im = Image.open(file_name)
        f, axarr = plt.subplots(1, 4, sharey=True, figsize=(20, 20))
        plt.cla()
        fid = 0
        for modelName in visAnno: #['prec_at_0.9']: #new_cls_thrsh_dict:
            print (modelName)
            ax = axarr[fid]
            visImageClsThrshs(plt, (1,4, fid+1), im, visAnno[modelName], ecolor=color_list[fid], title=modelName, showCateg=True)
            fid += 1
        if saveVis: # and len(visAnno['COCO_Mask_Detector']) > 0:
            plt.savefig(osp.join(visFolder,imgName), bbox_inches='tight',pad_inches = 0)
        plt.show()
        plt.close()

[31665, 55449, 55372, 79517, 102645, 96119, 77753, 109023, 72291, 4278, 96122, 35989, 65550, 63096, 99015, 39233, 3136, 104749, 65839, 88799, 6938, 28213, 71116, 75072, 76554, 22051, 72372, 41300, 90178, 109115, 97507, 83529, 2348, 112134, 101605, 56729, 38229, 69742, 68001, 4656, 120760, 1554, 27081, 26269, 62033, 120770, 5293, 64070, 70882, 73444, 84231, 35651, 62182, 12876, 9468, 92682, 50247, 106899, 19889, 81895, 19872, 69719, 102615, 56400, 99705, 33768, 73305, 54358, 72973, 111999, 79025, 68380, 61100, 105404, 79174, 119070, 4924, 77392, 79980, 39517, 507, 3420, 53030, 92114, 85804, 107631, 92181, 89465, 31937, 51322, 12144, 65529, 60923, 31269, 93603, 7373, 86615, 125377, 96210, 7122, 72244, 71397, 11233, 82498, 78030, 89912, 90876, 78552, 69654, 19988, 27618, 69268, 89355, 60869, 45886, 109006, 107159, 8523, 65673, 5290, 102182, 35535, 96615, 116528, 111496, 2796, 43400, 59128, 57862, 84308, 14722, 100446, 9685, 872, 55248, 66415, 64150, 67438, 13090, 86781, 71269, 115816, 799

Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'DA3EFBAD37D4651C7FF806201C609BCD3B9A9F4006D4B662C7D3493D9B13CEFFA818CAD39A71E70A7C2112D177F2CCA7EC1A5AC5B494E920ECE8B8180B3D1ABC.jpg', 'height': 760.0, 'width': 570.0, 'id': 77753}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:28 ==> GOD:359 home_or_office_furnishing_or_decor\frame_and_picture
======furniture:28 ==> GOD:359 home_or_office_furnishing_or_decor\frame_and_picture
========COCO:37 ==> GOD:507:sports_equipment\skating_or_roller\skateboard
furniture_GT
Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '0F0D3CE6EC1F4D8F6E7F081C2EBA369EC07BE2AE1DF5039245995BFC1EEAE3B0EE1877EEE43DB918C55329036DD1D452FA21169D9856D29C556B7C7A8647E1FE.jpg', 'height': 419.0, 'width': 280.0, 'id': 109023}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.typ

Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '8880C830AC678072AF18553868310028CA1D8315D9E468834ACA845D64D6E02D841848BF4D756B1742DF5934DB816803C4EB985FC23C18523290AA5982451FBE.jpg', 'height': 440.0, 'width': 550.0, 'id': 3136}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:49 ==> GOD:367 home_or_office_furnishing_or_decor\lighting\table_lamp
======furniture:55 ==> GOD:145 electronics\home_entertainment\tv
======furniture:26 ==> GOD:127 electronics\home_appliances\fireplace
======furniture:10 ==> GOD:390 home_or_office_furnishing_or_decor\storage\cabinet
======furniture:46 ==> GOD:397 home_or_office_furnishing_or_decor\storage\shelving
======furniture:50 ==> GOD:370 home_or_office_furnishing_or_decor\pillow_or_cushion\cushion
======furniture:12 ==> GOD:125 electronics\home_appliances\fan
======furniture:48 ==> GOD:380 home_or_office_furnishing_or_decor\seating\sofa
=

COCO_Mask_Detector
OpenImage_Detector
{'file_name': '1C0DD1FE63231092DDA9D939ABD5038EE49688FD5120CC6FFF0E73E2B9BA703622675939330E84F6F47F3FDE349A66101958A6D004D15A8EBCD8B8BFB1E8C87F.jpg', 'height': 550.0, 'width': 550.0, 'id': 71116}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:14 ==> GOD:379 home_or_office_furnishing_or_decor\seating\chair
======furniture:14 ==> GOD:379 home_or_office_furnishing_or_decor\seating\chair
======furniture:22 ==> GOD:401 home_or_office_furnishing_or_decor\table_or_desk
======furniture:3 ==> GOD:388 home_or_office_furnishing_or_decor\storage\basket
======furniture:41 ==> GOD:344 home_or_office_furnishing_or_decor\dinnerware_or_serveware\pitcher
======furniture:46 ==> GOD:397 home_or_office_furnishing_or_decor\storage\shelving
======furniture:39 ==> GOD:371 home_or_office_furnishing_or_decor\pillow_or_cushion\paper_decoration
======furniture:39 ==> GOD:371 hom

COCO_Mask_Detector
OpenImage_Detector
{'file_name': '7420A2C928A94F49688892F582A550E0BB81550C74E34B07DDBA3F379C7F64002752353E3F592F5C99C32D7FDC8A2791C94A937AE7E37262641594502F6FDC66.jpg', 'height': 736.0, 'width': 736.0, 'id': 109115}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
========= Open Image:pen ==> GOD:473:school_or_office_supply\pen
furniture_GT
Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'D8842BAA8674CBBC671BC801890B5081845A83FCF2EC0F50CCF3F7DBD06A969214CD9DF5413D5CE136C0691C091C0DC78AAA91D94D40DEA298E6F70368491DB8.jpg', 'height': 320.0, 'width': 240.0, 'id': 97507}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
furniture_GT
Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '5AB899D71C7A49AB77AD9C986913DF12511D03ED430348EAD8E7E8E7A071C7FE4A164A31353737D4BBE922344ADFB

...

Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'B9B027C9D510E21B17AF6E28C802BB1B1B8DEFBB4852C1DF678E5717790B19623D2F5F6A71FF72B2FBE545852A8DE5856BBB4CDD876403B4E2B2FEE12F279868.jpg', 'height': 650.0, 'width': 433.0, 'id': 23168}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
furniture_GT
Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '599C3A08410D01CD7E149303C7B7D5886B1003B6D34B1F87349154B5093ACE9C163E3B78B9178C2105BD6F131A463EB764996DF93B21EA6A6734BB5B7E713C0C.jpg', 'height': 1243.0, 'width': 540.0, 'id': 72220}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:28 ==> GOD:359 home_or_office_furnishing_or_decor\frame_and_picture
======furniture:11 ==> GOD:330 home_or_office_furnishing_or_decor\candle_or_candle_holder
======furniture:11 ==> GOD:330 home_or_office_furn

COCO_Mask_Detector
OpenImage_Detector
{'file_name': '4EEDED35D98C29ACB95B89458FF26B2A9E4A2A941BBAF441771F1DB0B87D29D49C8813425F33852A4C654CA99A9F140681E9417052D9FBFB0517BAB5D692873F.jpg', 'height': 360.0, 'width': 360.0, 'id': 124796}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:36 ==> GOD:368 home_or_office_furnishing_or_decor\mirror
======furniture:28 ==> GOD:359 home_or_office_furnishing_or_decor\frame_and_picture
======furniture:28 ==> GOD:359 home_or_office_furnishing_or_decor\frame_and_picture
======furniture:10 ==> GOD:390 home_or_office_furnishing_or_decor\storage\cabinet
======furniture:1 ==> GOD:376 home_or_office_furnishing_or_decor\seating\armchair
======furniture:42 ==> GOD:460 plant\potted_plant
======furniture:14 ==> GOD:379 home_or_office_furnishing_or_decor\seating\chair
======furniture:26 ==> GOD:127 electronics\home_appliances\fireplace
======furniture:56 ==> GOD:409 

Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'EEB29B86533BC312B007BC1777925E3C6465C3C4D65C10E35C50E888D0687FC3875176B89F574E724B6F5C99098CEB647E844A2052DFFECCDF111E146A37C5D9.jpg', 'height': 450.0, 'width': 338.0, 'id': 53446}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
furniture_GT
Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '403F3AB42432AE78DD80840DDAE60015CBA64D5ED6C22DF5270651F4D7A4552C44E4C348DC11270245DE07F591EC13FA1871509C64F96C379ECDAD1755A4CD76.jpg', 'height': 640.0, 'width': 427.0, 'id': 71797}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:49 ==> GOD:367 home_or_office_furnishing_or_decor\lighting\table_lamp
======furniture:56 ==> GOD:409 home_or_office_furnishing_or_decor\vase_or_pot
======furniture:28 ==> GOD:359 home_or_office_furnishing_or_d

Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '23A906A805D339855E3EFA0A904B861025ABFFE1D17FD84C2B30C747B6F25CA1AD786690C52F4A11E328F63A625275315E680E9807921A7D1A4D91835D49CC0F.jpg', 'height': 570.0, 'width': 570.0, 'id': 3138}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:28 ==> GOD:359 home_or_office_furnishing_or_decor\frame_and_picture
========COCO:17 ==> GOD:49:animal\mammal\dog
furniture_GT
Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'F0FE6B9D3ECFB78795CE12820C7075E63E730AE7CD7C5201A529F0D0BF86003B8911B3DDD7865CEAB6E4BE56AF70E19654926C28430800C853ECDDB3E66469C8.jpg', 'height': 282.0, 'width': 740.0, 'id': 3601}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:1 ==> GOD:376 home_or_office_furnishing_or_decor\seating\armchair
======furniture:

Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'B9F6BB39CED3C7A4AC6771B93A94D0EDAE704F06D1AE838A17D06B8521CFF2656AFF8ED0198BF52A842B71786C750CD0C9325C840D5471B9970AE40F20F92EEE.jpg', 'height': 165.0, 'width': 165.0, 'id': 109780}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:42 ==> GOD:460 plant\potted_plant
======furniture:46 ==> GOD:397 home_or_office_furnishing_or_decor\storage\shelving
======furniture:42 ==> GOD:460 plant\potted_plant
======furniture:3 ==> GOD:388 home_or_office_furnishing_or_decor\storage\basket
======furniture:56 ==> GOD:409 home_or_office_furnishing_or_decor\vase_or_pot
======furniture:42 ==> GOD:460 plant\potted_plant
======furniture:56 ==> GOD:409 home_or_office_furnishing_or_decor\vase_or_pot
======furniture:11 ==> GOD:330 home_or_office_furnishing_or_decor\candle_or_candle_holder
======furniture:28 ==> GOD:359 home_or_office_furnishing_or

In [11]:
# print (data['images'][:10])
# print ("Saving annotatinos to {}".format(outAnnoFile))
# with open(outAnnoFile,'w') as fout:
#     json.dump(data, fout)
# print ("Done")

[{'file_name': 'E3CCB304F9566D553A169B7279415E647902A57D5DEB4C1CA2E6939423381DCA0EC64DAB84F27F2D9A842E9C0C58BA58B7BBE36E34BE649E460A844E71D6B1E3.jpg', 'height': 733.0, 'width': 550.0, 'id': 31665}, {'file_name': '76FC7A5A958D86F7696689F40FAAE48AC7E5631307E63F51B25E26841FCCF86F6C48244AEBB6230096EF71323FC3B50BE1308BB9895756B24E424CF8C5B9BF44.jpg', 'height': 490.0, 'width': 736.0, 'id': 55449}, {'file_name': '78DB239735C6AD1931281C4F366F7692E8B096AEBDF2836FB292B41ECD644E0BD14540BCD628C16EF9A6D42AB998CFA80EADEBA0C3E1E485467F1FAFF90D2ECC.jpg', 'height': 768.0, 'width': 511.0, 'id': 55372}, {'file_name': 'E4E7B811946426F7D813F74DFB26EB883F48EF64A3D244C9261112B2C1BB79380663E92B894CE3BCE9DF9ECA43528F8248CB9A44F626FA3557B62CE8EF0FAD34.jpg', 'height': 363.0, 'width': 242.0, 'id': 79517}, {'file_name': '968B5C47AB681195EB993915BBA536FE00893D2028AB33C3C3880B8D0B83BFD442980835396B0C4A1A4BE4034D65054AD4B9AB4B86952082D98ADE2D1C9CA99B.jpg', 'height': 400.0, 'width': 327.0, 'id': 102645}, {'file_name':